# 車を検出し、その周りに境界ボックスを配置する

このノートブックでは、Yolo モデルによって検出された車の周囲にボックスを描く方法を学習します。

まず、YOLO パッケージをインポートします。

In [ ]:
# このラボ用に設計された Workbench イメージを使用していない場合は、次の行のコメントを解除して実行し、必要なパッケージをインストールできます。
# !pip install --no-cache-dir --no-dependencies -r requirements.txt

from ultralytics import YOLO
from PIL import Image

In [ ]:
# 物体検出にはYOLOv8mモデルを使用します。

model = YOLO("yolov8m.pt")

In [ ]:
# テスト画像でのモデル予測の結果を取得する

img = "images/carImage0.jpg"
results = model.predict(img)

In [ ]:
# YOLO を使用すると、1 枚だけではなく、複数の画像を送信でき、さまざまな結果が得られます。
# 送信した画像は 1 枚だけなので、結果の配列から最初の (そして唯一の) 要素のみを取得する必要があります。

result = results[0]

In [ ]:
# 見つかったボックスの数を検出する

len(result.boxes)

In [ ]:
# ボックスを分析する

box = result.boxes[0]
print("Object type:", box.cls)
print("Coordinates:", box.xyxy)
print("Probability:", box.conf)

In [ ]:
# テンソルから実際の値を解凍する

cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
# COCO は、YOLO モデルがトレーニングされたデータセットです。検出するようにトレーニングされたオブジェクトはクラスに編成されています。これは、"オブジェクト タイプ" フィールドで取得した情報です。
# YOLOv8 結果オブジェクトには、これらのクラスの「names」プロパティも含まれています。

print(result.names)


クラス番号「2」は「車」オブジェクトに対応していることがわかります。したがって、結果の境界ボックスは検出された「車」に対応しています。
画像の上にボックスを描いてみましょう！


In [ ]:
# まず、座標をリストに入れて丸めます。
# 次に、 result.names 辞書を使用して、検出されたオブジェクト クラスの名前を ID で取得します。

cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
# すべてのボックスをループして情報を抽出してみましょう。

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

In [ ]:
# 画像上に、ボックス、クラスの名前、確率（モデルが検出をどの程度確信しているか）を描画します。

Image.fromarray(result.plot()[:,:,::-1])

ここで、前回のノートブックでテストした複数の車の画像 (carImage4.jpg) に戻り、yolo が実際に画像内のすべての「車」を識別できるかどうかを確認しましょう。

In [ ]:
# これは前のセルで使用したのと同じコードですが、1回のパスで

results = model.predict("images/carImage4.jpg")

result = results[0]

for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")

Image.fromarray(result.plot()[:,:,::-1])

YOLO モデルは、画像の「奥」にある車をいくつか見逃していることがわかります。しかし、全体的には、モデルはこの画像内の複数の車をうまく識別しました。さらに重要なのは、識別された車が「境界ボックス」に囲まれていることがわかります。

YOLO モデルによって認識された車の周囲に境界ボックスを配置できるようになったので、YOLO モデルを再トレーニングして車の「衝突」を識別できるようになります。 

**ノートブック `04-03-model-retraining.ipynb` を開いてください。**